# Deep Q-Learning Algorithm
DQN 算法的伪代码如下，是Q-learning方法的扩展。
![](http://simplecore-dev.intel.com/ai/wp-content/uploads/sites/71/deep-Q-learning-algorithm.png)

详细的带入经验重放的DQN:
![](https://morvanzhou.github.io/static/results/reinforcement-learning/4-1-1.jpg)

In [8]:
import numpy as np
import tensorflow as tf

/home/louis/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
class DeepQNetwork():
    def __init__(
        self,
        n_features,
        n_actions,
        learning_rate = 0.01,
        reward_decay = 0.9,
        replace_target_iter = 300,
        e_greedy = 0.9,
        memory_size = 500,
        batch_size = 32,
        e_greedy_increment = False,
        output_graph = False
    ):
        self.n_features = n_features
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = reward_decay
        self.replace_target_iter = replace_target_iter
        self.e_greedy_max = e_greedy
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.e_greedy = 0 if e_greedy_increment is None else self.e_greedy_max # greedy值越大则explore效果越强
        
        self.lean_step_counters = 0
        
        self.memory = np.zeros((self.memory_size,self.n_features*2 +2))
        
        self._build_net()
        
        t_params = tf.get_collection('target_params')
        e_params = tf.get_collection('eval_params')
        
        self.replace_target_op = [tf.assign(t_param,e_param) for t_param, e_param in zip(t_params,e_params)]
        
        self.sess = tf.Session()
        
        tf.sess.run(tf.global_variables_initializer())
        
        if output_graph:
            tf.summary.FileWriter('./log/DQN',sess.graph)
            
        self.cost_his = []
        
    def _bulid_net(self):
#         evaluate_net
        self.s = tf.placeholder(tf.float32,[None,self.n_features],name = 's')
        self.q_target = tf.placeholder(tf.float32,[None,self.n_actions],name = 'Qtarget')
        with tf.variable_scope('eval_net'):
            cnames ,n_l1,w_initializer,b_initialier = ['eval_params',tf.GraphKeys.GLOBAL_VARIABLES],10
            ,tf.random_normal_initializer(0.,0.3), tf.constant_initializer(0.1)
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1',[self.n_features,n_l1],initializer=w_initializer,collections=cnames)
                b1 = tf.get_variable('b1',[1,n_l1],initializer=b_initialier,collections= cnames)
                l1 = tf.nn.relue(tf.matmul(self.s,w1) + b1)
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2',[n_l1,self.n_actions],initializer=w_initializer,collections=cnames)
                b2 = tf.get_variable('b2',[1,n_actions],initializer=b_initialier,collections=cnames)
                self.q_eval = tf.matmul(l1,w2) + b2
        with tf.variable_scope('loss'):
            self.loss = tf.losses.mean_squared_error(self.q_eval,self.q_target)
        with tf.variable_scope('train'):
            self._train_op = tf.train.RMSPropOptimizer(lr).minimize(self.loss)
        
#         target net
        self.s_ = tf.placeholder(tf.float32,[None,self.n_features],name = 's_')
        with tf.variable_scope('target_net'):
            cnames ,n_l1,w_initializer,b_initialier = ['target_params',tf.GraphKeys.GLOBAL_VARIABLES],10
            ,tf.random_normal_initializer(0.,0.3), tf.constant_initializer(0.1)
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1',[self.n_features,n_l1],initializer=w_initializer,collections=cnames)
                b1 = tf.get_variable('b1',[1,n_l1],initializer=b_initialier,collections= cnames)
                l1 = tf.nn.relue(tf.matmul(self.s,w1) + b1)
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2',[n_l1,self.n_actions],initializer=w_initializer,collections=cnames)
                b2 = tf.get_variable('b2',[1,n_actions],initializer=b_initialier,collections=cnames)
                self.q_next = tf.matmul(l1,w2) + b2
        

SyntaxError: invalid syntax (<ipython-input-15-20060607812a>, line 1)